In [14]:
import pandas as pd
import operator
import time
import numpy as np
import random
random.seed(0)


def dist(v1, v2):
    d = 0
    for n in range(0, len(v1)):
        d += (v1[n] - v2[n])**2
    d = d**0.5
    return d

def kN(v1, X, Y, r):
    d_list = []
    n_row = len(X.index)
    for n in range(0, n_row):
        d_list.append((Y.iloc[n], dist(v1, X.iloc[n])))
    neib_list = []
    for n in d_list:
        if n[1] <= r:
            neib_list.append(n)
    return neib_list

def LOO(data, r, neib_list_rad, max_class):
    missm = 0
    n_row = len(data.index)
    for n in range(0,n_row):
        neib_list_rad_red = []
        for h in neib_list_rad[n]:
            if h[1] < r:
                neib_list_rad_red.append(h[0])
        if neib_list_rad_red == []:
            u = max_class
        else:
            u = max(set(neib_list_rad_red), key = neib_list_rad_red.count)
        if neib_list_rad_red.count(u) == len(neib_list_rad_red)/2:
            u = max_class
        if u != data.iloc[n]['label']:
            missm += 1
    return missm/n_row

def step(data, b_l, b_r, f, g, neib_list_rad, max_class, d_max):
    s = (b_r-b_l)/4
    for r in [b_l, b_l+s, b_l+2*s, b_l+3*s, b_r]:
        f.append(r)
        g.append(LOO(data, r, neib_list_rad, max_class))
    b_l = int(max(0, f[np.argmin(g)]-s))
    b_r = int(min(f[np.argmin(g)]+s, d_max))
    return b_l, b_r, s, f[np.argmin(g)], min(g)

def main(data):
    X = data.drop('label', axis = 1)
    Y = data['label']
    max_class = max(set(list(Y)), key = list(Y).count)
    neib_list_rad = []
    d_max = 0
    for n in range(0, 20):
        d2 = dist(X.iloc[random.randint(0,len(data.index)-1)], X.iloc[random.randint(0,len(data.index)-1)])
        if d2 > d_max:
            d_max = d2
    for n in range(0, len(data.index)):
        neib_list_rad.append(kN(X.iloc[n], X.drop(n,axis = 0), Y.drop(n,axis = 0), d_max+1)) 
    f = []
    g = []
    b_l = 0
    b_r = d_max
    s = d_max
    while s > d_max/1000:
        ss = step(data, b_l, b_r, f, g, neib_list_rad, max_class, d_max)
        b_l, b_r, s = ss[:-2]

    print('r =', ss[3],', LOO =', ss[4])
    

In [21]:
from sklearn import preprocessing
n_sam = 200

t1 = time.time()
df = pd.read_csv('cancer.csv')
df = df.sample(n=n_sam).reset_index(drop=True)
print('cancer')
main(df)
print('time =', round((time.time()-t1)), 'sec\n')

t1 = time.time()
df = pd.read_csv('spam.csv')
df = df.sample(n=n_sam).reset_index(drop=True)           
print('spam')
main(df)
print('time =', round((time.time()-t1)), 'sec\n')


t1 = time.time()
df = pd.read_csv('cancer.csv')
df = df.sample(n=n_sam).reset_index(drop=True)
X = df.drop('label', axis=1)
X = pd.DataFrame(preprocessing.normalize(X))
Y = df['label']
df = pd.concat([X, Y], axis=1)
print('normalized cancer')
main(df)
print('time =', round((time.time()-t1)), 'sec\n')

t1 = time.time()
df = pd.read_csv('spam.csv')
df = df.sample(n=n_sam).reset_index(drop=True)
X = df.drop('label', axis=1)
X = pd.DataFrame(preprocessing.normalize(X))
Y = df['label']
df = pd.concat([X, Y], axis=1)
print('normalized spam')
main(df)
print('time =', round((time.time()-t1)), 'sec\n')

cancer
r = 379.75 , LOO = 0.11
time = 44 sec

spam
r = 676.0 , LOO = 0.325
time = 86 sec

normalized cancer
r = 0.049759047626446985 , LOO = 0.08
time = 32 sec

normalized spam
r = 0.23830839881892613 , LOO = 0.33
time = 61 sec

